**DVAMI20h**

- Arlind Iseni
- Alexander Jamal

## Assignment 2
The aim of Assignment 2 is to experimentally compare the computational and predictive performance of three learning algorithms on a spam detection task.

**Group assignment:** Max 2 students

**Prerequisite reading:** sections 12.1 - 12.3 in the main literature

**Language:** Python (Already implemented supervised learning algorithms and standard libraries can be used. However, It is NOT permitted to use any library or API that directly computes the Friedman and Nemeyi tests.)

**Data:** Spambase Dataset, https://archive.ics.uci.edu/ml/datasets/SpambaseLinks to an external site.

**Algorithms**  
three supervised classification learning algorithms of your choice.

**Evaluation measures:** perform a comparison between the selected algorithms based on 1) computational performance in terms of training time, 2) predictive performance based on accuracy, and 3) predictive performance based on F-measure.

**Procedure**  
(repeat steps 2, 3, and 4 for each evaluation measure above)

1. Run stratified ten-fold cross-validation tests.
2. Present the results exactly as in the table in example 12.4 of the main literature.
3. Conduct the Friedman test and report the results exactly as in the table in example 12.8 of the main literature.
4. Determine whether the average ranks as a whole display significant differences on the 0.05 alpha level and, if so, use the Nemeyi test to calculate the critical difference in order to determine which algorithms perform significantly different from each other.

**Compute**  
the size of possible instances
the size of hypothesis space (the number of possible extensions)
the number of possible conjunctive concepts according to the descriptions in Section 4.1 of the main literature
Implement the algorithm and verify that it works as expected.
Compute the accuracy of the model and report the generated model, i.e., the conjunctive rule.

**Written report**  
Template: The IEEE conference template and citation style should be followed (templatesLinks to an external site. in MS word and LaTeX).
Language: English without spelling mistakes.
Style: Clear.
Content: The report should give an overview of the conducted experiments and the obtained results. It should contain (but not be limited to) information about the used classifiers, a brief description of the Friedman and Nemeyi tests along with the formulas, results of the experiment as stated above, results of the comparison stating whether the algorithms perform significantly different or not from each other for each performance measure.
Format: PDF.
Page limit: 2 pages excluding references (no abstract should be included)

**Code**   
Provide meaningful comments for different blocks of the code. 
A README.TXT file must clearly state exactly how to execute the code and any necessary setups.

**Submission**  
Make sure to include your names in the report and the code.
The report must be submitted as a PDF separately (not to be included in the ZIP file).
Code and additional files related to implementation must be archived using ZIP.

### Import modules and dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import KBinsDiscretizer, Normalizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC

### Configure settings

In [3]:
%matplotlib widget
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 12)
plt.rcParams['figure.constrained_layout.use'] = True

### Load and read dataframe

In [4]:
# columns are saved in the data/names.txt file. Here we all entries without the newline character in a list.
with open("data/names.txt", "r") as f:
    columns = f.read().splitlines()

In [5]:
df = pd.read_csv("data/spambase.data", names=columns)

In [6]:
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_orders,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


### Data exploration

In [7]:
df["is_spam"].value_counts(normalize=True)

0    0.605955
1    0.394045
Name: is_spam, dtype: float64

### Data cleaning

#### null-values

In [8]:
df.isna().sum().any()

False

#### duplicates

In [9]:
df.duplicated().sum()

391

#### negative values

In [10]:
(df < 0).all().sum()

0

### Split data

In [11]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [12]:
skf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)

### Data transformation

In [13]:
def fitter(X_: pd.DataFrame, transformer_, params: dict):
    """
    Fits an algorithm to data.
    
    Args:
        X_: pandas dataframe (unlabeled data)
        transformer_: any algorithm used for data transformation
        params: parameters used to initialize transformer function
    
    Returns:
        transformation function object
    """
    return transformer_(**params).fit(X_)

In [14]:
def transformer(X_: pd.DataFrame, fitter_: fitter) -> pd.DataFrame:
    """
    Transforms dataframe.
    
    Args:
        X_: pandas data frame (unlabeled data)
        fitter_: the fit we use to transform
    
    Returns:
        pandas dataframe
    """
    return pd.DataFrame(fitter_.transform(X_))

### Instantiation

In [38]:
svm_clf = SVC()
ada_clf = AdaBoostClassifier()
rf_clf = RandomForestClassifier()

### Friedman test

https://medium.com/mlearning-ai/comparing-classifiers-friedman-and-nemenyi-tests-32294103ee12

#### Stratified K-fold

In [22]:
def stratified(X_: pd.DataFrame, y_: pd.Series, clf, transformer_=None, params=None) -> tuple[np.array]:
    """
    loops through kfold stratified training / test data splits and trains model.
    
    Args:
        X_: pandas data frame (unlabeled data)
        y_: pandas series (label data)
        clf: classifier algorithm
        transformer_: what algorithm for transformation (optional)
        params: dictionary with arguments for the transformer
    
    Returns:
        tuple of reshaped np arrays (row arrays)
    """
    f1 = []
    for train_index, test_index in skf.split(X, y):
        params = {} if params == None else params
        fit = None if transformer_ == None else fitter(X.iloc[train_index], transformer_, params)
        X_train = X_.iloc[train_index] if fit == None else transformer(X_.iloc[train_index], fit)
        X_test = X_.iloc[test_index] if fit == None else transformer(X_.iloc[test_index], fit)
        
        y_train, y_test = y_[train_index], y_[test_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        f1.append(f1_score(y_pred, y_test))
        
    return f1

In [23]:
def frame_template(models=["Support Vector Machine Classifier", "AdaBoost Classifier", "Random Forest Classifier"]) -> pd.DataFrame:
    return pd.DataFrame(columns=models, index=[i for i in range(1, 11)]+["Average"])

In [100]:
f1 = frame_template()

In [101]:
f1

,Support Vector Machine Classifier,AdaBoost Classifier,Random Forest Classifier
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN


In [220]:
kbins_params = dict(n_bins=7, encode="ordinal", strategy="kmeans")
for col, model in zip(f1.columns, [svm_clf, ada_clf, rf_clf]):
    if model != svm_clf:
        f1.loc[f1.index[:-1], col] = stratified(X, y, model)  
    else:
        f1.loc[f1.index[:-1], col] = stratified(X, y, svm_clf, transformer_=KBinsDiscretizer, params=kbins_params)
    f1.iloc[-1, :] = 0

In [221]:
f1

,Support Vector Machine Classifier,AdaBoost Classifier,Random Forest Classifier
1,0.901408,0.92,0.940845
2,0.913165,0.936288,0.935933
3,0.906516,0.916905,0.913793
4,0.925714,0.919668,0.931429
5,0.926554,0.943503,0.949153
6,0.932249,0.93617,0.942779
7,0.932945,0.928367,0.960674
8,0.928367,0.956044,0.96648
9,0.865014,0.820253,0.873684
10,0.835735,0.815642,0.817927


In [258]:
f1_scores = f1.rank(axis=1, method="max", ascending=False)

In [260]:
f1_scores.loc["Average"] = f1_scores.iloc[:-1, :].mean()

In [267]:
f1_scores = f1_scores.astype(str)

In [268]:
f1_scores

,Support Vector Machine Classifier,AdaBoost Classifier,Random Forest Classifier
1,3.0,2.0,1.0
2,3.0,1.0,2.0
3,3.0,1.0,2.0
4,2.0,3.0,1.0
5,3.0,2.0,1.0
6,3.0,2.0,1.0
7,2.0,3.0,1.0
8,3.0,2.0,1.0
9,2.0,3.0,1.0
10,1.0,3.0,2.0


In [269]:
f1["Support Vector Machine Classifier"] = f1["Support Vector Machine Classifier"].astype(str).str.cat("(" + f1_scores["Support Vector Machine Classifier"] + ")", sep =" ")
f1["AdaBoost Classifier"] = f1["AdaBoost Classifier"].astype(str).str.cat("(" + f1_scores["AdaBoost Classifier"] + ")", sep =" ")
f1["Random Forest Classifier"] = f1["Random Forest Classifier"].astype(str).str.cat("(" + f1_scores["Random Forest Classifier"] + ")", sep =" ")
f1.loc["Average"] = f1_scores.loc["Average"]

In [270]:
f1

,Support Vector Machine Classifier,AdaBoost Classifier,Random Forest Classifier
1,0.9014084507042254 (3.0),0.9199999999999999 (2.0),0.9408450704225353 (1.0)
2,0.9131652661064426 (3.0),0.9362880886426593 (1.0),0.9359331476323121 (2.0)
3,0.9065155807365438 (3.0),0.9169054441260746 (1.0),0.9137931034482759 (2.0)
4,0.9257142857142856 (2.0),0.9196675900277009 (3.0),0.9314285714285714 (1.0)
5,0.9265536723163841 (3.0),0.943502824858757 (2.0),0.9491525423728814 (1.0)
6,0.9322493224932249 (3.0),0.9361702127659575 (2.0),0.9427792915531336 (1.0)
7,0.9329446064139941 (2.0),0.9283667621776505 (3.0),0.9606741573033707 (1.0)
8,0.9283667621776505 (3.0),0.956043956043956 (2.0),0.9664804469273743 (1.0)
9,0.8650137741046833 (2.0),0.8202531645569621 (3.0),0.8736842105263157 (1.0)
10,0.835734870317003 (1.0),0.8156424581005587 (3.0),0.8179271708683474 (2.0)
